In [1]:
import json
from langchain.agents import Tool, AgentExecutor, ZeroShotAgent
from langchain.llms import Ollama
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

class PDFSearchTool:
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore

    def search(self, query: str) -> str:
        """Search the vector store for relevant information"""
        results = self.vectorstore.similarity_search(query, k=2)
        return "\n".join(doc.page_content for doc in results)

def load_pdf_and_create_vector_store(pdf_path):
    """Load PDF and create vector store"""
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    embeddings = OllamaEmbeddings(model="deepseek-llm:7b-chat")
    return FAISS.from_documents(docs, embeddings)

def create_tools(vectorstore):
    pdf_tool = PDFSearchTool(vectorstore)
    return [
        Tool(
            name="PDF_Search",
            func=pdf_tool.search,
            description="Search the PDF document for relevant information. Use this tool when you need specific information from the document."
        )
    ]

# Updated prompt template with clearer instructions
template = """Use the following format:

Question: {input}
Thought: Consider if you need to search the PDF for specific information
Action: If needed, use the PDF_Search tool to find relevant information
Observation: Note what you found or why you didn't need to search
Answer: Provide your answer in this JSON format:
{{
    "answer": "concise answer here",
    "explanation": "detailed explanation here"
}}

Start working on the question:
Question: {input}
{agent_scratchpad}
"""

def setup_agent(vectorstore):
    # Initialize components
    llm = Ollama(
        model="deepseek-llm:7b-chat",
        temperature=0.7
    )
    
    # Create prompt
    prompt = PromptTemplate(
        template=template,
        input_variables=["input", "agent_scratchpad"]
    )
    
    # Create chain
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    # Create tools
    tools = create_tools(vectorstore)
    
    # Create agent
    agent = ZeroShotAgent(
        llm_chain=llm_chain,
        tools=tools,
        verbose=True
    )
    
    # Create executor with reasonable limits
    return AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        verbose=True,
        max_iterations=3,  # Limit iterations to prevent infinite loops
        max_execution_time=30,  # 30 second timeout
        handle_parsing_errors=True
    )

def extract_json_from_text(text: str) -> dict:
    """Extract JSON from text and handle common formatting issues"""
    try:
        # Find JSON content
        start = text.find('{')
        end = text.rfind('}') + 1
        if start >= 0 and end > 0:
            json_str = text[start:end]
            return json.loads(json_str)
    except:
        # If extraction fails, format the text as JSON
        return {
            "answer": text[:200] if len(text) > 200 else text,
            "explanation": text
        }

def query_agent(question: str, executor: AgentExecutor) -> dict:
    """Query the agent with a question"""
    try:
        # Execute the query
        response = executor({"input": question})
        
        # Handle different response types
        if isinstance(response, dict) and "output" in response:
            return extract_json_from_text(response["output"])
        elif isinstance(response, str):
            return extract_json_from_text(response)
        return response
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return {
            "answer": "Error occurred while processing the query",
            "explanation": str(e)
        }

# Example usage
if __name__ == "__main__":
    # Initialize vector store
    pdf_path = "English.pdf"
    vectorstore = load_pdf_and_create_vector_store(pdf_path)
    
    # Setup agent
    agent_executor = setup_agent(vectorstore)
    
    # Test the system
    query = "What is the role of counselling in preventing drug addiction?"
    result = query_agent(query, agent_executor)
    print(json.dumps(result, indent=2))

/var/folders/_x/8clrdsj55jv7dc605rcxqx3r0000gn/T/ipykernel_2877/522778108.py:23: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="deepseek-llm:7b-chat")
/var/folders/_x/8clrdsj55jv7dc605rcxqx3r0000gn/T/ipykernel_2877/522778108.py:56: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(
/var/folders/_x/8clrdsj55jv7dc605rcxqx3r0000gn/T/ipykerne



> Entering new AgentExecutor chain...
{
    "answer": "Counselling can play a crucial role in preventing drug addiction by offering individuals emotional support, guidance, and resources to address their substance use disorder.",
    "explanation": "Through counseling sessions, individuals struggling with drug addiction can receive personalized interventions that aim at helping them understand the underlying causes of their addiction. Counselors help clients develop coping mechanisms for dealing with stress and other triggers that may prompt a relapse. Additionally, they provide education on the dangers of substance abuse to create awareness about its consequences.\n\nMoreover, counseling sessions often involve connecting individuals with treatment options like rehabilitation centers, therapy groups, or medication-assisted treatments (MATs). In this way, it helps clients navigate through available resources and find a suitable path for recovery. It is essential to note that the effec